In [1]:
!pip install gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time 

# Import API key
from api_keys import g_key

# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:

weather_data = pd.read_csv("../starter_code/cities.csv")
weather_data




,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Toccoa,90,US,1590103827,56,34.58,-83.33,73.40,4.70
1,Kapaa,90,US,1590103827,83,22.08,-159.32,77.00,21.92
2,Chui,100,UY,1590103827,95,-33.70,-53.46,62.28,11.61
3,Punta Arenas,75,CL,1590103612,75,-53.15,-70.92,44.60,12.75
4,Barrhead,100,GB,1590103827,71,55.80,-4.39,57.00,8.05
5,Bambous Virieux,75,MU,1590103827,73,-20.34,57.76,75.20,18.34
6,Yellowknife,90,CA,1590103719,46,62.46,-114.35,50.00,11.41
7,Hilo,20,US,1590103827,58,19.73,-155.09,84.20,8.05
8,Cape Town,20,ZA,1590103535,93,-33.93,18.42,57.99,2.24
9,Alma,75,CA,1590103711,33,48.55,-71.65,79.00,18.34


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key) 

In [5]:
humidity = weather_data["Humidity"]

In [6]:
locations = weather_data[["Lat", "Lng"]]

In [7]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)

# Display figure
fig


TraitError: The 'max_intensity' trait of a WeightedHeatmap instance must be a float, but a value of 100 <class 'numpy.int64'> was specified.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
narrow_cities_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrow_cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
156,Padang,0,ID,1590103841,69,-0.95,100.35,79.86,4.09
193,Jiexiu,0,CN,1590103844,22,37.02,111.91,71.29,3.71
228,Riyadh,0,SA,1590103715,15,24.69,46.72,78.80,6.93
366,Abu Samrah,0,SY,1590103860,25,35.30,37.18,78.04,6.02
371,Buraidah,0,SA,1590103603,31,26.33,43.98,77.00,4.70
382,Nioro,0,GM,1590103861,58,13.35,-15.75,74.68,7.16
522,Maricá,0,BR,1590103874,73,-22.92,-42.82,75.20,3.36
524,Mossamedes,0,AO,1590103874,58,-15.20,12.15,75.56,3.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:

#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
# Create a hotel_df
hotel_df = narrow_cities_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.


for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 156: Padang.
Closest hotel in Padang is Mercure Padang.
------------
Retrieving Results for Index 193: Jiexiu.
Closest hotel in Jiexiu is Jinyuan Hotel.
------------
Retrieving Results for Index 228: Riyadh.
Closest hotel in Riyadh is Hyatt Regency Riyadh Olaya.
------------
Retrieving Results for Index 366: Abu Samrah.
Missing field/result... skipping.
------------
Retrieving Results for Index 371: Buraidah.
Closest hotel in Buraidah is Ewaa Express Hotel - Buraydah.
------------
Retrieving Results for Index 382: Nioro.
Missing field/result... skipping.
------------
Retrieving Results for Index 522: Maricá.
Closest hotel in Maricá is Pousada Caribe Carioca (021)96713-9711.
------------
Retrieving Results for Index 524: Mossamedes.
Closest hotel in Mossamedes is Hotel ibis Styles Iu Namibe.
------------
-------End of Search-------


In [11]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
156,Padang,ID,-0.95,100.35,Mercure Padang
193,Jiexiu,CN,37.02,111.91,Jinyuan Hotel
228,Riyadh,SA,24.69,46.72,Hyatt Regency Riyadh Olaya
366,Abu Samrah,SY,35.30,37.18,
371,Buraidah,SA,26.33,43.98,Ewaa Express Hotel - Buraydah
382,Nioro,GM,13.35,-15.75,
522,Maricá,BR,-22.92,-42.82,Pousada Caribe Carioca (021)96713-9711
524,Mossamedes,AO,-15.20,12.15,Hotel ibis Styles Iu Namibe


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
# Display figure

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))